In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error
import datetime
from tqdm import tqdm
import time

In [6]:
firm = pd.read_csv("../data/raw/mma_sample_v2.csv")
firm.head(2)

,date,ret_eom,permno,shrcd,exchcd,mspread,year,month,size_port,rf,...,betadown_252d,bidaskhl_21d,corr_1260d,betabab_1260d,rmax5_rvol_21d,age,qmj,qmj_prof,qmj_growth,qmj_safety
0,20000131,20000131,10078,11,3,0.017178,2000,1,B,0.0041,...,2.219037,0.012635,0.504688,1.581540,1.373224,180,1.700939,1.711756,1.476410,1.087626
1,20000131,20000131,10104,11,3,0.019720,2000,1,B,0.0041,...,1.437330,0.016634,0.473872,1.941648,2.058353,180,0.828930,1.717670,-0.603531,0.608215


In [2]:
# === Step 1: Load and merge data ===
# Load firm-level data
firm = pd.read_csv("../data/raw/mma_sample_v2.csv", parse_dates=["date", "ret_eom"])

# Convert ret_eom (return date) to yyyymm for alignment
firm["yyyymm"] = firm["ret_eom"].dt.to_period("M").dt.to_timestamp()
firm["year"] = firm["yyyymm"].dt.year
firm["month"] = firm["yyyymm"].dt.month

# Load macroeconomic variables and surprise predictions
macro = pd.read_csv("../data/macro/macro_monthly.csv")
macro["yyyymm"] = pd.to_datetime(macro["yyyymm"].astype(str), format="%Y%m")

surprise = pd.read_csv("../results/surprise_predictions.csv", parse_dates=["date"])
surprise["yyyymm"] = surprise["date"].dt.to_period("M").dt.to_timestamp()
surprise = surprise.rename(columns={"predicted_surprise": "surprise_pred"})

# Load list of stock characteristic features
factor_list = pd.read_csv("../data/raw/factor_char_list.csv")["variable"].tolist()

In [3]:
# Filter firm data to include only relevant features
firm = firm[["permno", "yyyymm", "stock_exret", "year", "month"] + factor_list]

# Drop observations with missing target value
firm = firm[firm["stock_exret"].notna()].copy()

# Merge predicted surprise
firm = firm.merge(surprise, on=["permno", "yyyymm"], how="left")

# Merge macro variables
df_full = firm.merge(macro, on="yyyymm", how="left")

In [18]:
df_full.head(2)

,permno,yyyymm,stock_exret,age,aliq_at,aliq_mat,ami_126d,at_be,at_gr1,at_me,...,date,pred_surprise,dp,ep,bm,ntis,svar,dfy,tms,tbl
0,10078,2000-01-01,0.010428,180,1.1216,0.182049,0.000051,1.749945,0.474393,0.069654,...,NaT,NaN,-4.423938,-3.346471,0.154654,0.025359,0.005206,0.0055,0.0134,0.0532
1,10104,2000-01-01,-0.112577,180,0.9079,0.117486,0.000083,1.798574,0.156012,0.041240,...,NaT,NaN,-4.423938,-3.346471,0.154654,0.025359,0.005206,0.0055,0.0134,0.0532


In [4]:
# === Step 2: Create interaction terms in batches (excluding surprise_pred and date) ===
macro_vars = ['dp', 'ep', 'ntis', 'bm', 'svar', 'dfy', 'tms', 'tbl']
interaction_terms = []
batch_size = 20  # adjust as needed depending on memory capacity

for i in range(0, len(factor_list), batch_size):
    batch_vars = factor_list[i:i + batch_size]
    interaction_data = {}

    for macro_var in macro_vars:
        for firm_var in batch_vars:
            col_name = f"{firm_var}_{macro_var}"
            interaction_data[col_name] = df_full[firm_var] * df_full[macro_var]
            interaction_terms.append(col_name)

    interaction_df = pd.DataFrame(interaction_data)
    df_full = pd.concat([df_full.reset_index(drop=True), interaction_df.reset_index(drop=True)], axis=1)
    del interaction_df, interaction_data  # free memory after each batch

In [5]:
# Add surprise_pred as a direct feature (no interaction)
if "surprise_pred" not in df_full.columns:
    df_full["surprise_pred"] = np.nan

# Final feature list for modeling
all_features = factor_list + interaction_terms + ["surprise_pred"]

In [6]:
# === Step 3: Monthly standardization with median imputation ===
# Ensure all expected features exist before standardization
for col in all_features:
    if col not in df_full.columns:
        df_full[col] = np.nan

data = pd.DataFrame()
monthly = df_full.groupby("yyyymm")
for date, group in monthly:
    group = group.copy()
    for var in all_features:
        median_val = group[var].median(skipna=True)
        group[var] = group[var].fillna(median_val)
        group[var] = group[var].rank(method="dense") - 1
        vmax = group[var].max()
        group[var] = (group[var] / vmax) * 2 - 1 if vmax > 0 else 0
    data = pd.concat([data, group], ignore_index=True)

C:\Users\Yuriii\.venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Yuriii\.venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Yuriii\.venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Yuriii\.venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Yuriii\.venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Yuriii\.venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Y

In [7]:
# === Chunked Standardization Function ===
def chunk_standardize(df_train, df_val, df_test, columns, chunk_size=100):
    for i in range(0, len(columns), chunk_size):
        subset = columns[i:i+chunk_size]
        scaler = StandardScaler().fit(df_train[subset])
        df_train.loc[:, subset] = scaler.transform(df_train[subset])
        df_val.loc[:, subset] = scaler.transform(df_val[subset])
        df_test.loc[:, subset] = scaler.transform(df_test[subset])
    return df_train, df_val, df_test

## OLS

In [9]:
# === Step 4: Expanding window OLS-only training and prediction ===
data['date'] = data['yyyymm']
start_round = 0
starting = pd.to_datetime("20000101", format="%Y%m%d")
counter = start_round

print("Starting expanding window prediction (OLS only)...")

# Count total rounds based on full expansion logic
end_date = pd.to_datetime("20240101", format="%Y%m%d")
total_rounds = 0
while (starting + pd.DateOffset(years=11 + total_rounds)) <= end_date:
    total_rounds += 1

pbar = tqdm(total=total_rounds - start_round, desc="OLS Rounds")

while (counter < total_rounds):
    round_start_time = time.time()

    cutoff = [
        starting,
        starting + pd.DateOffset(years=8 + counter),
        starting + pd.DateOffset(years=10 + counter),
        starting + pd.DateOffset(years=11 + counter),
    ]

    print(f"\n=== Round {counter + 1}: {cutoff[0].strftime('%Y-%m')} to {cutoff[3].strftime('%Y-%m')} ===")

    train = data[(data["date"] >= cutoff[0]) & (data["date"] < cutoff[1])]
    validate = data[(data["date"] >= cutoff[1]) & (data["date"] < cutoff[2])]
    test = data[(data["date"] >= cutoff[2]) & (data["date"] < cutoff[3])]

    print(f"Train size: {train.shape}, Val size: {validate.shape}, Test size: {test.shape}")

    if test.empty or train.empty or validate.empty:
        print(f"Skipping round {counter + 1} due to empty set.")
        counter += 1
        pbar.update(1)
        continue

    print("Standardizing features (chunked)...")
    train, validate, test = chunk_standardize(train, validate, test, all_features, chunk_size=100)
    print("Standardization complete.")

    X_train, Y_train = train[all_features].values, train['stock_exret'].values
    X_test, Y_test = test[all_features].values, test['stock_exret'].values

    Y_mean = np.mean(Y_train)
    Y_train_dm = Y_train - Y_mean

    reg_pred = test[["permno", "yyyymm", "stock_exret"]].copy()

    print("Training OLS...")
    reg = LinearRegression(fit_intercept=False)
    reg.fit(X_train, Y_train_dm)
    reg_pred["ols"] = reg.predict(X_test) + Y_mean

    r2_ols = 1 - np.sum((reg_pred["ols"] - Y_test) ** 2) / np.sum((Y_test - Y_test.mean()) ** 2)

    # Append predictions and R² per round
    reg_pred.to_csv("../results/ols_predictions_all.csv", mode="a", header=(counter==start_round), index=False)
    pd.DataFrame([{
        "round": counter + 1,
        "start_date": cutoff[0],
        "end_date": cutoff[3],
        "r2_ols": r2_ols
    }]).to_csv("../results/ols_r2_all.csv", mode="a", header=(counter==start_round), index=False)

    print(f"Round {counter + 1} completed. R²: {r2_ols:.4f}, Time: {round(time.time() - round_start_time, 2)}s")
    counter += 1
    pbar.update(1)

print("OLS-only prediction complete.")

Starting expanding window prediction (OLS only)...



OLS Rounds:  50%|███████████████████████████████████                                   | 7/14 [20:23<20:23, 174.77s/it]


=== Round 1: 2000-01 to 2011-01 ===


Train size: (93496, 1339), Val size: (20897, 1339), Test size: (10563, 1339)
Standardizing features (chunked)...
Standardization complete.
Training OLS...



 Rounds:   7%|█████                                                                  | 1/14 [00:37<08:08, 37.59s/it]

...

Round 12 completed. R²: -0.0011, Time: 92.47s

=== Round 13: 2000-01 to 2023-01 ===
Train size: (223615, 1339), Val size: (24239, 1339), Test size: (13192, 1339)
Standardizing features (chunked)...
Standardization complete.
Training OLS...



 Rounds:  93%|█████████████████████████████████████████████████████████████████     | 13/14 [14:04<01:23, 83.66s/it]

Round 13 completed. R²: -0.0274, Time: 95.03s

=== Round 14: 2000-01 to 2024-01 ===
Train size: (235344, 1339), Val size: (25702, 1339), Test size: (12327, 1339)
Standardizing features (chunked)...
Standardization complete.
Training OLS...



 Rounds: 100%|██████████████████████████████████████████████████████████████████████| 14/14 [15:44<00:00, 88.48s/it]

Round 14 completed. R²: -0.0808, Time: 99.55s
OLS-only prediction complete.


## Lasso

In [10]:
# === Step 4: Expanding window Lasso-only training and prediction ===
data['date'] = data['yyyymm']
start_round = 0
starting = pd.to_datetime("20000101", format="%Y%m%d")
counter = start_round

print("Starting expanding window prediction (Lasso only)...")

# Determine total number of rounds
end_date = pd.to_datetime("20240101", format="%Y%m%d")
total_rounds = 0
temp_start = starting
while (temp_start + pd.DateOffset(years=11 + total_rounds)) <= end_date:
    total_rounds += 1

pbar = tqdm(total=total_rounds - start_round, desc="Lasso Rounds")

while (counter < total_rounds):
    round_start_time = time.time()

    cutoff = [
        starting,
        starting + pd.DateOffset(years=8 + counter),
        starting + pd.DateOffset(years=10 + counter),
        starting + pd.DateOffset(years=11 + counter),
    ]

    print(f"\n=== Round {counter + 1}: {cutoff[0].strftime('%Y-%m')} to {cutoff[3].strftime('%Y-%m')} ===")

    train = data[(data["date"] >= cutoff[0]) & (data["date"] < cutoff[1])]
    validate = data[(data["date"] >= cutoff[1]) & (data["date"] < cutoff[2])]
    test = data[(data["date"] >= cutoff[2]) & (data["date"] < cutoff[3])]

    print(f"Train size: {train.shape}, Val size: {validate.shape}, Test size: {test.shape}")

    if test.empty or train.empty or validate.empty:
        print(f"Skipping round {counter + 1} due to empty set.")
        counter += 1
        pbar.update(1)
        continue

    print("Standardizing features (chunked)...")
    train, validate, test = chunk_standardize(train, validate, test, all_features, chunk_size=100)
    print("Standardization complete.")

    X_train, Y_train = train[all_features].values, train['stock_exret'].values
    X_val, Y_val = validate[all_features].values, validate['stock_exret'].values
    X_test, Y_test = test[all_features].values, test['stock_exret'].values

    Y_mean = np.mean(Y_train)
    Y_train_dm = Y_train - Y_mean

    reg_pred = test[["permno", "yyyymm", "stock_exret"]].copy()

    print("Tuning Lasso alpha...")
    lambdas = np.arange(-4, 4.1, 0.1)
    val_mse = np.zeros(len(lambdas))
    for ind, i in enumerate(tqdm(lambdas, desc="Alpha Grid")):
        reg = Lasso(alpha=10**i, max_iter=1000000, fit_intercept=False)
        reg.fit(X_train, Y_train_dm)
        val_mse[ind] = mean_squared_error(Y_val, reg.predict(X_val) + Y_mean)
    best_lambda = lambdas[np.argmin(val_mse)]

    print(f"Best lambda: 10^{best_lambda:.2f}")
    reg = Lasso(alpha=10**best_lambda, max_iter=1000000, fit_intercept=False)
    reg.fit(X_train, Y_train_dm)
    reg_pred["lasso"] = reg.predict(X_test) + Y_mean

    r2_lasso = 1 - np.sum((reg_pred["lasso"] - Y_test) ** 2) / np.sum((Y_test - Y_test.mean()) ** 2)

    # Save prediction and r2 per round
    reg_pred.to_csv("../results/lasso_predictions_all.csv", mode="a", header=(counter==start_round), index=False)
    pd.DataFrame([{
        "round": counter + 1,
        "start_date": cutoff[0],
        "end_date": cutoff[3],
        "r2_lasso": r2_lasso
    }]).to_csv("../results/lasso_r2_all.csv", mode="a", header=(counter==start_round), index=False)

    print(f"Round {counter + 1} completed. R²: {r2_lasso:.4f}, Time: {round(time.time() - round_start_time, 2)}s")
    counter += 1
    pbar.update(1)

print("Lasso-only prediction complete.")

Starting expanding window prediction (Lasso only)...


OLS Rounds: 100%|█████████████████████████████████████████████████████████████████████| 14/14 [28:20<00:00, 121.49s/it]


=== Round 1: 2000-01 to 2011-01 ===


Train size: (93496, 1339), Val size: (20897, 1339), Test size: (10563, 1339)
Standardizing features (chunked)...
Standardization complete.
Tuning Lasso alpha...



Alpha Grid: 100%|██████████████████████████████████████████████████████████████████████| 81/81 [06:13<00:00,  4.62s/it]


...

Round 13 completed. R²: -0.0299, Time: 1321.91s

=== Round 14: 2000-01 to 2024-01 ===
Train size: (235344, 1339), Val size: (25702, 1339), Test size: (12327, 1339)
Standardizing features (chunked)...
Standardization complete.
Tuning Lasso alpha...



Alpha Grid: 100%|██████████████████████████████████████████████████████████████████████| 81/81 [25:34<00:00, 18.95s/it]


Best lambda: 10^-3.40


Lasso Rounds: 100%|████████████████████████████████████████████████████████████████| 14/14 [2:49:32<00:00, 1138.05s/it]

Round 14 completed. R²: -0.0104, Time: 1620.04s
Lasso-only prediction complete.


In [23]:
import pandas as pd
import numpy as np

lambda_df = pd.read_csv("../results/best_lambda_by_round.csv")
lambda_raw = lambda_df["Best Lambda (Alpha)"].astype(str)

lambda_df.head(2)

,Round,Best Lambda (Alpha)
0,1,10^-2.40
1,2,10^-2.20


### Lasso Feature Selection

In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from collections import Counter
from tqdm import tqdm

# === Step 0: Load and parse lambda values correctly ===
lambda_df = pd.read_csv("../results/best_lambda_by_round.csv", skiprows=1, header=None)

# This will convert "10^-2.4" → 10 ** -2.4 → 0.00398 (numeric alpha)
def parse_lambda(expr):
    try:
        exponent = float(expr.split("^")[1])
        return 10 ** exponent
    except:
        return np.nan

best_lambdas = lambda_df.iloc[:, 1].apply(parse_lambda).dropna().values

# === Step 1: Initialize ===
feature_counter = Counter()
data['date'] = data['yyyymm']
starting = pd.to_datetime("20000101", format="%Y%m%d")

# === Step 2: Loop through all rounds ===
for counter in tqdm(range(len(best_lambdas)), desc="Refitting Lasso"):
    cutoff = [
        starting,
        starting + pd.DateOffset(years=8 + counter),
        starting + pd.DateOffset(years=10 + counter),
        starting + pd.DateOffset(years=11 + counter),
    ]

    train = data[(data["date"] >= cutoff[0]) & (data["date"] < cutoff[1])]
    validate = data[(data["date"] >= cutoff[1]) & (data["date"] < cutoff[2])]

    if train.empty or validate.empty:
        print(f"Skipping round {counter + 1} due to empty set.")
        continue

    X_train = train[all_features].values
    Y_train = train['stock_exret'].values
    Y_mean = np.mean(Y_train)
    Y_train_dm = Y_train - Y_mean

    # alpha is already numeric
    alpha = best_lambdas[counter]
    reg = Lasso(alpha=alpha, max_iter=1000000, fit_intercept=False)
    reg.fit(X_train, Y_train_dm)

    nonzero_mask = reg.coef_ != 0
    nonzero_vars = np.array(all_features)[nonzero_mask]
    feature_counter.update(nonzero_vars)

    print(f"Round {counter+1}: alpha={alpha:.5f}, nonzero features={len(nonzero_vars)}")

# === Step 4: Output top feature frequency ===
freq_df = pd.DataFrame.from_dict(feature_counter, orient='index', columns=['count'])
freq_df = freq_df.sort_values('count', ascending=False)
freq_df.to_csv("../results/lasso_top_features.csv")

print(f"Finished! Total selected features: {len(freq_df)}")
print("Top feature frequency saved to ../results/lasso_top_features.csv")

Refitting Lasso:   7%|████▋                                                             | 1/14 [00:06<01:24,  6.47s/it]

Round 1: alpha=0.00398, nonzero features=3


Refitting Lasso:  14%|█████████▍                                                        | 2/14 [00:13<01:19,  6.66s/it]

Round 2: alpha=0.00631, nonzero features=0


Refitting Lasso:  21%|██████████████▏                                                   | 3/14 [00:21<01:19,  7.19s/it]

...

Refitting Lasso:  93%|████████████████████████████████████████████████████████████▎    | 13/14 [04:45<00:35, 35.79s/it]

Round 13: alpha=0.00398, nonzero features=0


Refitting Lasso: 100%|█████████████████████████████████████████████████████████████████| 14/14 [05:33<00:00, 23.79s/it]

Round 14: alpha=0.00040, nonzero features=129


Finished! Total selected features: 131
Top feature frequency saved to ../results/lasso_top_features.csv


## OLS Reduced Feature

In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import time

# === Load reduced features ===
reduced_features = pd.read_csv("../results/lasso_top_features.csv")
reduced_feature_list = reduced_features.iloc[:, 0].tolist()

# === Chunked Standardization Function ===
def chunk_standardize(df_train, df_val, df_test, columns, chunk_size=100):
    for i in range(0, len(columns), chunk_size):
        subset = columns[i:i+chunk_size]
        scaler = StandardScaler().fit(df_train[subset])
        df_train.loc[:, subset] = scaler.transform(df_train[subset])
        df_val.loc[:, subset] = scaler.transform(df_val[subset])
        df_test.loc[:, subset] = scaler.transform(df_test[subset])
    return df_train, df_val, df_test

# === Step 4: Expanding window OLS (Reduced) ===
data['date'] = data['yyyymm']
start_round = 0
starting = pd.to_datetime("20000101", format="%Y%m%d")
counter = start_round

print("Starting expanding window prediction (OLS with reduced features)...")

# Determine total number of rounds
end_date = pd.to_datetime("20240101", format="%Y%m%d")
total_rounds = 0
temp_start = starting
while (temp_start + pd.DateOffset(years=11 + total_rounds)) <= end_date:
    total_rounds += 1

pbar = tqdm(total=total_rounds - start_round, desc="OLS Rounds")

while (counter < total_rounds):
    round_start_time = time.time()

    cutoff = [
        starting,
        starting + pd.DateOffset(years=8 + counter),
        starting + pd.DateOffset(years=10 + counter),
        starting + pd.DateOffset(years=11 + counter),
    ]

    print(f"\n=== Round {counter + 1}: {cutoff[0].strftime('%Y-%m')} to {cutoff[3].strftime('%Y-%m')} ===")

    train = data[(data["date"] >= cutoff[0]) & (data["date"] < cutoff[1])]
    validate = data[(data["date"] >= cutoff[1]) & (data["date"] < cutoff[2])]
    test = data[(data["date"] >= cutoff[2]) & (data["date"] < cutoff[3])]

    print(f"Train size: {train.shape}, Val size: {validate.shape}, Test size: {test.shape}")

    if test.empty or train.empty or validate.empty:
        print(f"Skipping round {counter + 1} due to empty set.")
        counter += 1
        pbar.update(1)
        continue

    print("Standardizing reduced features (chunked)...")
    train, validate, test = chunk_standardize(train, validate, test, reduced_feature_list, chunk_size=100)
    print("Standardization complete.")

    X_train = train[reduced_feature_list].values
    Y_train = train['stock_exret'].values
    X_test = test[reduced_feature_list].values
    Y_test = test['stock_exret'].values

    Y_mean = np.mean(Y_train)
    Y_train_dm = Y_train - Y_mean

    reg_pred = test[["permno", "yyyymm", "stock_exret"]].copy()

    print("Training OLS...")
    reg = LinearRegression(fit_intercept=False)
    reg.fit(X_train, Y_train_dm)
    reg_pred["ols"] = reg.predict(X_test) + Y_mean

    r2_ols = 1 - np.sum((reg_pred["ols"] - Y_test) ** 2) / np.sum((Y_test - Y_test.mean()) ** 2)

    # Save prediction and r2 per round
    reg_pred.to_csv("../results/ols_predictions_reduced.csv", mode="a", header=(counter==start_round), index=False)
    pd.DataFrame([{
        "round": counter + 1,
        "start_date": cutoff[0],
        "end_date": cutoff[3],
        "r2_ols": r2_ols
    }]).to_csv("../results/ols_r2_reduced.csv", mode="a", header=(counter==start_round), index=False)

    print(f"Round {counter + 1} completed. R²: {r2_ols:.4f}, Time: {round(time.time() - round_start_time, 2)}s")
    counter += 1
    pbar.update(1)

print("OLS (Reduced) prediction complete.")

Starting expanding window prediction (OLS with reduced features)...


Ridge Rounds: 100%|████████████████████████████████████████████████████████████████████| 14/14 [12:14<00:00, 52.48s/it]


=== Round 1: 2000-01 to 2011-01 ===


Train size: (93496, 1339), Val size: (20897, 1339), Test size: (10563, 1339)
Standardizing reduced features (chunked)...
Standardization complete.
Training OLS...


OLS Rounds:   7%|█████                                                                  | 1/14 [00:12<02:39, 12.27s/it]

...

Round 12 completed. R²: 0.0027, Time: 27.88s

=== Round 13: 2000-01 to 2023-01 ===
Train size: (223615, 1339), Val size: (24239, 1339), Test size: (13192, 1339)
Standardizing reduced features (chunked)...
Standardization complete.
Training OLS...


OLS Rounds:  93%|█████████████████████████████████████████████████████████████████     | 13/14 [03:43<00:25, 25.06s/it]

Round 13 completed. R²: -0.0262, Time: 29.02s

=== Round 14: 2000-01 to 2024-01 ===
Train size: (235344, 1339), Val size: (25702, 1339), Test size: (12327, 1339)
Standardizing reduced features (chunked)...
Standardization complete.
Training OLS...


OLS Rounds: 100%|██████████████████████████████████████████████████████████████████████| 14/14 [04:14<00:00, 26.98s/it]

Round 14 completed. R²: -0.0157, Time: 31.42s
OLS (Reduced) prediction complete.


## XGBoost Reduced Feature

In [17]:
# === Step 4: Expanding window XGBoost (Reduced) ===
data['date'] = data['yyyymm']
start_round = 0
starting = pd.to_datetime("20000101", format="%Y%m%d")
counter = start_round

print("Starting expanding window prediction (XGBoost with reduced features)...")

# Determine total number of rounds
end_date = pd.to_datetime("20240101", format="%Y%m%d")
total_rounds = 0
temp_start = starting
while (temp_start + pd.DateOffset(years=11 + total_rounds)) <= end_date:
    total_rounds += 1

pbar = tqdm(total=total_rounds - start_round, desc="XGBoost Rounds")

while (counter < total_rounds):
    round_start_time = time.time()

    cutoff = [
        starting,
        starting + pd.DateOffset(years=8 + counter),
        starting + pd.DateOffset(years=10 + counter),
        starting + pd.DateOffset(years=11 + counter),
    ]

    print(f"\n=== Round {counter + 1}: {cutoff[0].strftime('%Y-%m')} to {cutoff[3].strftime('%Y-%m')} ===")

    train = data[(data["date"] >= cutoff[0]) & (data["date"] < cutoff[1])]
    validate = data[(data["date"] >= cutoff[1]) & (data["date"] < cutoff[2])]
    test = data[(data["date"] >= cutoff[2]) & (data["date"] < cutoff[3])]

    print(f"Train size: {train.shape}, Val size: {validate.shape}, Test size: {test.shape}")

    if test.empty or train.empty or validate.empty:
        print(f"Skipping round {counter + 1} due to empty set.")
        counter += 1
        pbar.update(1)
        continue

    X_train = train[reduced_feature_list].values
    Y_train = train['stock_exret'].values
    X_val = validate[reduced_feature_list].values
    Y_val = validate['stock_exret'].values
    X_test = test[reduced_feature_list].values
    Y_test = test['stock_exret'].values

    Y_mean = np.mean(Y_train)
    Y_train_dm = Y_train - Y_mean

    reg_pred = test[["permno", "yyyymm", "stock_exret"]].copy()

    print("Training XGBoost...")
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    model.fit(X_train, Y_train_dm)

    reg_pred["xgb"] = model.predict(X_test) + Y_mean

    r2_xgb = 1 - np.sum((reg_pred["xgb"] - Y_test) ** 2) / np.sum((Y_test - Y_test.mean()) ** 2)

    # Save prediction and r2 per round
    reg_pred.to_csv("../results/xgb_predictions_reduced.csv", mode="a", header=(counter==start_round), index=False)
    pd.DataFrame([{
        "round": counter + 1,
        "start_date": cutoff[0],
        "end_date": cutoff[3],
        "r2_xgb": r2_xgb
    }]).to_csv("../results/xgb_r2_reduced.csv", mode="a", header=(counter==start_round), index=False)

    print(f"Round {counter + 1} completed. R²: {r2_xgb:.4f}, Time: {round(time.time() - round_start_time, 2)}s")
    counter += 1
    pbar.update(1)

print("XGBoost (Reduced) prediction complete.")

Starting expanding window prediction (XGBoost with reduced features)...



XGBoost Rounds:  14%|█████████▌                                                         | 2/14 [01:49<10:58, 54.87s/it]


=== Round 1: 2000-01 to 2011-01 ===


Train size: (93496, 1339), Val size: (20897, 1339), Test size: (10563, 1339)
Training XGBoost...



oost Rounds:   7%|████▊                                                              | 1/14 [00:21<04:36, 21.27s/it]

...

Round 12 completed. R²: -0.0519, Time: 52.46s

=== Round 13: 2000-01 to 2023-01 ===
Train size: (223615, 1339), Val size: (24239, 1339), Test size: (13192, 1339)
Training XGBoost...



oost Rounds:  93%|█████████████████████████████████████████████████████████████▎    | 13/14 [08:08<00:51, 51.66s/it]

Round 13 completed. R²: -0.0454, Time: 64.58s

=== Round 14: 2000-01 to 2024-01 ===
Train size: (235344, 1339), Val size: (25702, 1339), Test size: (12327, 1339)
Training XGBoost...



oost Rounds: 100%|██████████████████████████████████████████████████████████████████| 14/14 [09:07<00:00, 54.01s/it]

Round 14 completed. R²: -0.0814, Time: 59.43s
XGBoost (Reduced) prediction complete.


## Elastic Net Reduced Feature

In [19]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

# === Load 131 reduced features ===
reduced_features = pd.read_csv("../results/lasso_top_features.csv")
reduced_feature_list = reduced_features.iloc[:, 0].tolist()

data['date'] = data['yyyymm']
starting = pd.to_datetime("20000101", format="%Y%m%d")
start_round = 0
counter = start_round

print("Starting expanding window prediction (ElasticNet simplified)...")

# Determine total number of rounds
end_date = pd.to_datetime("20240101", format="%Y%m%d")
total_rounds = 0
temp_start = starting
while (temp_start + pd.DateOffset(years=11 + total_rounds)) <= end_date:
    total_rounds += 1

pbar = tqdm(total=total_rounds - start_round, desc="ElasticNet Rounds")

while (counter < total_rounds):
    round_start_time = time.time()

    cutoff = [
        starting,
        starting + pd.DateOffset(years=8 + counter),
        starting + pd.DateOffset(years=10 + counter),
        starting + pd.DateOffset(years=11 + counter),
    ]

    print(f"\n=== Round {counter + 1}: {cutoff[0].strftime('%Y-%m')} to {cutoff[3].strftime('%Y-%m')} ===")

    train = data[(data["date"] >= cutoff[0]) & (data["date"] < cutoff[1])]
    validate = data[(data["date"] >= cutoff[1]) & (data["date"] < cutoff[2])]
    test = data[(data["date"] >= cutoff[2]) & (data["date"] < cutoff[3])]

    print(f"Train size: {train.shape}, Val size: {validate.shape}, Test size: {test.shape}")

    if test.empty or train.empty or validate.empty:
        print(f"Skipping round {counter + 1} due to empty set.")
        counter += 1
        pbar.update(1)
        continue

    print("Standardizing features (chunked)...")
    train, validate, test = chunk_standardize(train, validate, test, reduced_feature_list, chunk_size=100)
    print("Standardization complete.")

    X_train = train[reduced_feature_list].values
    Y_train = train['stock_exret'].values
    X_val = validate[reduced_feature_list].values
    Y_val = validate['stock_exret'].values
    X_test = test[reduced_feature_list].values
    Y_test = test['stock_exret'].values

    Y_mean = np.mean(Y_train)
    Y_train_dm = Y_train - Y_mean

    reg_pred = test[["permno", "yyyymm", "stock_exret"]].copy()

    print("Tuning ElasticNet (alpha only)...")
    lambdas = np.arange(-4, 4.1, 0.1)
    val_mse = np.zeros(len(lambdas))
    for ind, i in enumerate(lambdas):
        reg = ElasticNet(alpha=(10**i), l1_ratio=0.5, max_iter=1000000, fit_intercept=False)
        reg.fit(X_train, Y_train_dm)
        val_mse[ind] = mean_squared_error(Y_val, reg.predict(X_val) + Y_mean)

    best_lambda = lambdas[np.argmin(val_mse)]
    reg = ElasticNet(alpha=(10**best_lambda), l1_ratio=0.5, max_iter=1000000, fit_intercept=False)
    reg.fit(X_train, Y_train_dm)
    reg_pred["enet_simple"] = reg.predict(X_test) + Y_mean

    r2_enet = 1 - np.sum((reg_pred["enet_simple"] - Y_test) ** 2) / np.sum((Y_test - Y_test.mean()) ** 2)

    # Save predictions and R2
    reg_pred.to_csv("../results/enet_predictions_reduced_simple.csv", mode="a", header=(counter==start_round), index=False)
    pd.DataFrame([{
        "round": counter + 1,
        "start_date": cutoff[0],
        "end_date": cutoff[3],
        "r2_enet_simple": r2_enet
    }]).to_csv("../results/enet_r2_reduced_simple.csv", mode="a", header=(counter==start_round), index=False)

    print(f"Round {counter + 1} completed. R²: {r2_enet:.4f}, Time: {round(time.time() - round_start_time, 2)}s")
    counter += 1
    pbar.update(1)

print("ElasticNet simplified prediction complete.")

Starting expanding window prediction (ElasticNet simplified)...



ElasticNet Rounds:   7%|████▎                                                       | 1/14 [17:52<3:52:27, 1072.91s/it]


=== Round 1: 2000-01 to 2011-01 ===


Train size: (93496, 1339), Val size: (20897, 1339), Test size: (10563, 1339)
Standardizing features (chunked)...
Standardization complete.
Tuning ElasticNet (alpha only)...



sticNet Rounds:   7%|████▎                                                        | 1/14 [15:20<3:19:27, 920.58s/it]

...

Round 12 completed. R²: -0.0076, Time: 1284.86s

=== Round 13: 2000-01 to 2023-01 ===
Train size: (223615, 1339), Val size: (24239, 1339), Test size: (13192, 1339)
Standardizing features (chunked)...
Standardization complete.
Tuning ElasticNet (alpha only)...



sticNet Rounds:  93%|██████████████████████████████████████████████████████▊    | 13/14 [3:53:59<21:18, 1278.45s/it]

Round 13 completed. R²: -0.0256, Time: 1587.01s

=== Round 14: 2000-01 to 2024-01 ===
Train size: (235344, 1339), Val size: (25702, 1339), Test size: (12327, 1339)
Standardizing features (chunked)...
Standardization complete.
Tuning ElasticNet (alpha only)...



sticNet Rounds: 100%|███████████████████████████████████████████████████████████| 14/14 [4:22:40<00:00, 1411.85s/it]

Round 14 completed. R²: -0.0092, Time: 1720.09s
ElasticNet simplified prediction complete.


## Ridge

In [10]:
# === 04_predict_return_ridge_reduced.ipynb ===

import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import datetime
import time

# === Load reduced features ===
reduced_features = pd.read_csv("../results/lasso_top_features.csv")
reduced_feature_list = reduced_features.iloc[:, 0].tolist()

# === Chunked Standardization Function ===
def chunk_standardize(df_train, df_val, df_test, columns, chunk_size=100):
    for i in range(0, len(columns), chunk_size):
        subset = columns[i:i+chunk_size]
        scaler = StandardScaler().fit(df_train[subset])
        df_train.loc[:, subset] = scaler.transform(df_train[subset])
        df_val.loc[:, subset] = scaler.transform(df_val[subset])
        df_test.loc[:, subset] = scaler.transform(df_test[subset])
    return df_train, df_val, df_test

# === Step 4: Expanding window Ridge (Reduced) ===
data['date'] = data['yyyymm']
start_round = 0
starting = pd.to_datetime("20000101", format="%Y%m%d")
counter = start_round

print("Starting expanding window prediction (Ridge with reduced features)...")

# Determine total number of rounds
end_date = pd.to_datetime("20240101", format="%Y%m%d")
total_rounds = 0
temp_start = starting
while (temp_start + pd.DateOffset(years=11 + total_rounds)) <= end_date:
    total_rounds += 1

pbar = tqdm(total=total_rounds - start_round, desc="Ridge Rounds")

while (counter < total_rounds):
    round_start_time = time.time()

    cutoff = [
        starting,
        starting + pd.DateOffset(years=8 + counter),
        starting + pd.DateOffset(years=10 + counter),
        starting + pd.DateOffset(years=11 + counter),
    ]

    print(f"\n=== Round {counter + 1}: {cutoff[0].strftime('%Y-%m')} to {cutoff[3].strftime('%Y-%m')} ===")

    train = data[(data["date"] >= cutoff[0]) & (data["date"] < cutoff[1])]
    validate = data[(data["date"] >= cutoff[1]) & (data["date"] < cutoff[2])]
    test = data[(data["date"] >= cutoff[2]) & (data["date"] < cutoff[3])]

    print(f"Train size: {train.shape}, Val size: {validate.shape}, Test size: {test.shape}")

    if test.empty or train.empty or validate.empty:
        print(f"Skipping round {counter + 1} due to empty set.")
        counter += 1
        pbar.update(1)
        continue

    print("Standardizing reduced features (chunked)...")
    train, validate, test = chunk_standardize(train, validate, test, reduced_feature_list, chunk_size=100)
    print("Standardization complete.")

    X_train, Y_train = train[reduced_feature_list].values, train['stock_exret'].values
    X_val, Y_val = validate[reduced_feature_list].values, validate['stock_exret'].values
    X_test, Y_test = test[reduced_feature_list].values, test['stock_exret'].values

    Y_mean = np.mean(Y_train)
    Y_train_dm = Y_train - Y_mean

    reg_pred = test[["permno", "yyyymm", "stock_exret"]].copy()

    print("Tuning Ridge alpha...")
    lambdas = np.arange(-1, 8.1, 0.1)
    val_mse = np.zeros(len(lambdas))
    for ind, i in enumerate(tqdm(lambdas, desc="Ridge Alpha Grid")):
        reg = Ridge(alpha=(10**i) * 0.5, fit_intercept=False)
        reg.fit(X_train, Y_train_dm)
        val_mse[ind] = mean_squared_error(Y_val, reg.predict(X_val) + Y_mean)
    best_lambda = lambdas[np.argmin(val_mse)]
    best_alpha = (10**best_lambda) * 0.5

    print(f"Best Ridge alpha: {best_alpha:.6f}")
    reg = Ridge(alpha=best_alpha, fit_intercept=False)
    reg.fit(X_train, Y_train_dm)
    reg_pred["ridge"] = reg.predict(X_test) + Y_mean

    r2_ridge = 1 - np.sum((reg_pred["ridge"] - Y_test) ** 2) / np.sum((Y_test - Y_test.mean()) ** 2)

    # Save prediction and r2 per round
    reg_pred.to_csv("../results/ridge_predictions_reduced.csv", mode="a", header=(counter==start_round), index=False)
    pd.DataFrame([{
        "round": counter + 1,
        "start_date": cutoff[0],
        "end_date": cutoff[3],
        "r2_ridge": r2_ridge,
        "best_alpha": best_alpha
    }]).to_csv("../results/ridge_r2_reduced.csv", mode="a", header=(counter==start_round), index=False)

    print(f"Round {counter + 1} completed. R²: {r2_ridge:.4f}, Time: {round(time.time() - round_start_time, 2)}s")
    counter += 1
    pbar.update(1)

print("Ridge (Reduced) prediction complete.")

Starting expanding window prediction (Ridge with reduced features)...



Ridge Rounds:   0%|                                                                             | 0/14 [01:28<?, ?it/s]


=== Round 1: 2000-01 to 2011-01 ===


Train size: (93496, 1339), Val size: (20897, 1339), Test size: (10563, 1339)
Standardizing reduced features (chunked)...
Standardization complete.
Tuning Ridge alpha...


Ridge Alpha Grid: 100%|████████████████████████████████████████████████████████████████| 91/91 [00:14<00:00,  6.38it/s]



...

Round 13 completed. R²: -0.0246, Time: 62.62s

=== Round 14: 2000-01 to 2024-01 ===
Train size: (235344, 1339), Val size: (25702, 1339), Test size: (12327, 1339)
Standardizing reduced features (chunked)...
Standardization complete.
Tuning Ridge alpha...


Ridge Alpha Grid: 100%|████████████████████████████████████████████████████████████████| 91/91 [00:36<00:00,  2.51it/s]


Best Ridge alpha: 500000.000000



ge Rounds: 100%|████████████████████████████████████████████████████████████████████| 14/14 [09:04<00:00, 58.10s/it]

Round 14 completed. R²: -0.0038, Time: 72.14s
Ridge (Reduced) prediction complete.


In [7]:
# === Step 4: Expanding window model training and prediction ===
data['date'] = data['yyyymm']
starting = pd.to_datetime("20000101", format="%Y%m%d")
counter = 0
pred_out = pd.DataFrame()
r2_records = []

print("Starting expanding window prediction...")

while (starting + pd.DateOffset(years=11 + counter)) <= pd.to_datetime("20240101", format="%Y%m%d"):
    print(f"\n=== Round {counter + 1}: {starting.strftime('%Y-%m')} to {(starting + pd.DateOffset(years=11 + counter)).strftime('%Y-%m')} ===")

    cutoff = [
        starting,
        starting + pd.DateOffset(years=8 + counter),
        starting + pd.DateOffset(years=10 + counter),
        starting + pd.DateOffset(years=11 + counter),
    ]

    train = data[(data["date"] >= cutoff[0]) & (data["date"] < cutoff[1])]
    validate = data[(data["date"] >= cutoff[1]) & (data["date"] < cutoff[2])]
    test = data[(data["date"] >= cutoff[2]) & (data["date"] < cutoff[3])]

    print(f"Train size: {train.shape}, Val size: {validate.shape}, Test size: {test.shape}")

    if test.empty or train.empty or validate.empty:
        print(f"Skipping round {counter + 1} due to empty set.")
        counter += 1
        continue

    print("Standardizing features...")
    scaler = StandardScaler().fit(train[all_features])
    train.loc[:, all_features] = scaler.transform(train[all_features])
    validate.loc[:, all_features] = scaler.transform(validate[all_features])
    test.loc[:, all_features] = scaler.transform(test[all_features])
    print("Standardization complete.")

    X_train, Y_train = train[all_features].values, train['stock_exret'].values
    X_val, Y_val = validate[all_features].values, validate['stock_exret'].values
    X_test, Y_test = test[all_features].values, test['stock_exret'].values

    Y_mean = np.mean(Y_train)
    Y_train_dm = Y_train - Y_mean

    reg_pred = test[["permno", "yyyymm", "stock_exret"]].copy()

    # === OLS ===
    print("Training OLS...")
    reg = LinearRegression(fit_intercept=False)
    reg.fit(X_train, Y_train_dm)
    reg_pred["ols"] = reg.predict(X_test) + Y_mean
    r2_ols = 1 - np.sum((reg_pred["ols"] - Y_test) ** 2) / np.sum((Y_test - Y_test.mean()) ** 2)

    # === Lasso ===
    print("Tuning Lasso alpha...")
    lambdas = np.arange(-4, 4.1, 0.1)
    val_mse = np.zeros(len(lambdas))
    for ind, i in enumerate(tqdm(lambdas, desc="Lasso alpha")):
        reg = Lasso(alpha=10**i, max_iter=1000000, fit_intercept=False)
        reg.fit(X_train, Y_train_dm)
        val_mse[ind] = mean_squared_error(Y_val, reg.predict(X_val) + Y_mean)
    best_lambda = lambdas[np.argmin(val_mse)]
    reg = Lasso(alpha=10**best_lambda, max_iter=1000000, fit_intercept=False)
    reg.fit(X_train, Y_train_dm)
    reg_pred["lasso"] = reg.predict(X_test) + Y_mean
    r2_lasso = 1 - np.sum((reg_pred["lasso"] - Y_test) ** 2) / np.sum((Y_test - Y_test.mean()) ** 2)

    # === Ridge ===
    print("Tuning Ridge alpha...")
    lambdas = np.arange(-1, 8.1, 0.1)
    val_mse = np.zeros(len(lambdas))
    for ind, i in enumerate(tqdm(lambdas, desc="Ridge alpha")):
        reg = Ridge(alpha=(10**i) * 0.5, fit_intercept=False)
        reg.fit(X_train, Y_train_dm)
        val_mse[ind] = mean_squared_error(Y_val, reg.predict(X_val) + Y_mean)
    best_lambda = lambdas[np.argmin(val_mse)]
    reg = Ridge(alpha=(10**best_lambda) * 0.5, fit_intercept=False)
    reg.fit(X_train, Y_train_dm)
    reg_pred["ridge"] = reg.predict(X_test) + Y_mean
    r2_ridge = 1 - np.sum((reg_pred["ridge"] - Y_test) ** 2) / np.sum((Y_test - Y_test.mean()) ** 2)

    # === ElasticNet (with early stopping and reduced grid) ===
    print("Tuning ElasticNet alpha (fast mode)...")
    lambdas = np.arange(-3, 2.25, 0.25)  # reduced grid
    val_mse = np.zeros(len(lambdas))
    for ind, i in enumerate(tqdm(lambdas, desc="ElasticNet alpha")):
        reg = ElasticNet(alpha=10**i, max_iter=5000, tol=1e-3, fit_intercept=False)
        reg.fit(X_train, Y_train_dm)
        val_mse[ind] = mean_squared_error(Y_val, reg.predict(X_val) + Y_mean)
    best_lambda = lambdas[np.argmin(val_mse)]
    reg = ElasticNet(alpha=10**best_lambda, max_iter=5000, tol=1e-3, fit_intercept=False)
    reg.fit(X_train, Y_train_dm)
    reg_pred["en"] = reg.predict(X_test) + Y_mean
    r2_en = 1 - np.sum((reg_pred["en"] - Y_test) ** 2) / np.sum((Y_test - Y_test.mean()) ** 2)

    # Save R^2 per model per round
    r2_records.append({
        "round": counter + 1,
        "start_date": cutoff[0],
        "end_date": cutoff[3],
        "r2_ols": r2_ols,
        "r2_lasso": r2_lasso,
        "r2_ridge": r2_ridge,
        "r2_en": r2_en
    })

    pred_out = pd.concat([pred_out, reg_pred], ignore_index=True)
    counter += 1

Starting expanding window prediction...

=== Round 1: 2000-01 to 2011-01 ===
Train size: (93496, 1339), Val size: (20897, 1339), Test size: (10563, 1339)
Standardizing features...
Standardization complete.
Training OLS...
Tuning Lasso alpha...


Lasso alpha: 100%|█████████████████████████████████████████████████████████████████████| 81/81 [06:45<00:00,  5.01s/it]


Tuning Ridge alpha...


Ridge alpha: 100%|█████████████████████████████████████████████████████████████████████| 91/91 [04:18<00:00,  2.84s/it]


Tuning ElasticNet alpha (fast mode)...


...

Ridge alpha: 100%|█████████████████████████████████████████████████████████████████████| 91/91 [08:58<00:00,  5.92s/it]


Tuning ElasticNet alpha (fast mode)...


ElasticNet alpha: 100%|████████████████████████████████████████████████████████████████| 21/21 [07:51<00:00, 22.44s/it]



=== Round 11: 2000-01 to 2021-01 ===
Train size: (201347, 1339), Val size: (22268, 1339), Test size: (11729, 1339)
Standardizing features...


MemoryError: Unable to allocate 1.99 GiB for an array with shape (201347, 1324) and data type float64

In [ ]:
# Save predictions and R^2 records
pred_out.to_csv("../data/results/stock_return_predictions.csv", index=False)
pd.DataFrame(r2_records).to_csv("../data/processed/model_r2_each_round.csv", index=False)